In [9]:
import yfinance as yf
from datetime import datetime
import warnings

# Suprimir avisos específicos
warnings.filterwarnings("ignore", category=FutureWarning, module="yfinance")


In [2]:
# Criar objeto do ticker
ticker_data = yf.Ticker("MSFT")

In [ ]:
#### Não Funciona ####
# # Acessar fast_info
info = ticker_data.info

In [6]:
yahoo_symbol_recommendations = ticker_data.recommendations
# yahoo_symbol_recommendations

In [8]:
# Dividendos
cashflow = ticker_data.cashflow
# cashflow

In [9]:
# Notícias
balance_sheet = ticker_data.balance_sheet
# balance_sheet

In [10]:
yahoo_symbol_sustainability = ticker_data.sustainability
# yahoo_symbol_sustainability

In [9]:
# Eventos corporativos
calendar = ticker_data.calendar
# calendar

In [10]:
# # Datas disponíveis para opções
# options_dates = ticker_data.options
# print("Datas de opções:", options_dates)

# # Obter opções para uma data específica
# options = ticker_data.option_chain(options_dates[0])
# print("Opções de compra (calls):", options.calls)
# print("Opções de venda (puts):", options.puts)

In [11]:
# # Demonstrativo financeiro
# financials = ticker_data.financials
# print("Demonstrativo financeiro:", financials)

# # Balanço patrimonial
# balance_sheet = ticker_data.balance_sheet
# print("Balanço patrimonial:", balance_sheet)

# # Fluxo de caixa
# cashflow = ticker_data.cashflow
# print("Fluxo de caixa:", cashflow)

Informações financeiras mais detalhadas da empresa.

In [12]:
# # Dados de sustentabilidade
# sustainability = ticker_data.sustainability
# print(sustainability)

Informações de ESG (Environmental, Social, Governance).

In [13]:
# # Ações em circulação
# shares = ticker_data.shares
# print("Ações em circulação:", shares)

Obtenha informações sobre ações em circulação e histórico de splits.

In [14]:
# Principais investidores institucionais
inst_holders = ticker_data.institutional_holders
# inst_holders


Dados sobre investidores institucionais e sua participação.

In [15]:
# Principais investidores institucionais
holders = ticker_data.major_holders
# holders


In [10]:
def get_yahoo_data_history(symbol : str, period : str, interval: str, start = '1900-01-01', end = datetime.now(), prepost : bool = True):
        '''
        Data collection from yahoo

        Parameters:
        period : str
            Valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max Either Use period parameter or use start and end
        interval : str
            Valid intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo Intraday data cannot extend last 60 days
        start: str
            Download start date string (YYYY-MM-DD) or _datetime, inclusive. Default is 1900-01-01 E.g. for start="2020-01-01", the first data point will be on "2020-01-01"
        end: str
            Download end date string (YYYY-MM-DD) or _datetime, exclusive. Default is now E.g. for end="2023-01-01", the last data point will be on "2022-12-31"
        prepost : bool
            Include Pre and Post market data in results? Default is False
        debug: bool
            If passed as False, will suppress error message printing to console.
        '''
        yahoo_data_history = yf.Ticker(symbol).history(period=period, interval=interval, start=start, end=end, prepost=prepost)
        return yahoo_data_history

history = get_yahoo_data_history(symbol="MSFT",period='1y', interval='1d', end = datetime.now(), prepost=True)

In [14]:
import requests

# Define the endpoint and your API key
url = "https://api.simplywall.st/graphql"
headers = {
    "Authorization": "Bearer sws:ZGE4YzBlNzItMTg2Ni00Y2Y5LWI3YTktYzAxODcyZGEwZjE3OmFjODUzODRhZDUzNGMwMzY=",
    "Content-Type": "application/json",  # Required for JSON body
}

# Define the GraphQL query and variables
query = """
query Companies($exchange: String!, $offset: Int!, $limit: Int!) {
  companies(exchange: $exchange, offset: $offset, limit: $limit) {
    id
    exchangeSymbol
    tickerSymbol
    name
    marketCapUSD
    primaryIndustry { name }
    secondaryIndustry { name }
    tertiaryIndustry { name }
    market { name iso2 }
    closingPrices
    statements { name title area type value outcome description state severity outcomeName }
    listings { id exchangeSymbol tickerSymbol name marketCapUSD primaryIndustry { name } secondaryIndustry { name } tertiaryIndustry { name } market { name iso2 } closingPrices }
    owners { name type sharesHeld percentOfSharesOutstanding holdingDate periodStartDate periodEndDate }
    insiderTransactions { type ownerName ownerType description tradeDateMin tradeDateMax shares priceMin priceMax transactionValue percentageSharesTraded }
    members { age name title tenure compensation }
    active classificationStatus
  }
}
"""

# Define the variables for the query
variables = {
    "exchange": "NYSE",  # Try changing to "NYSE" or another exchange
    "offset": 20,         # Try adjusting this value (e.g., 10 or 20)
    "limit": 100          # Adjust limit to fetch more results
}

# Create the request payload
payload = {
    "query": query,
    "variables": variables
}

# Make the POST request
response = requests.post(url, headers=headers, json=payload)

# Check the response
if response.status_code == 200:
    data = response.json()
    if not data:
        print("No companies found with the given parameters.")
    else:
        print("Returned Data:", data)
else:
    print(f"Error: {response.status_code}, {response.text}")


Returned Data: {'data': {'companies': []}}


In [1]:
import os
import subprocess
import requests

class TALibInstaller:
    def install_ta_lib_windows(self):
        """Automatiza a instalação do TA-Lib no Windows."""
        print("Iniciando instalação do TA-Lib no Windows...")

        # URL do arquivo .whl (atualize conforme necessário)
        talib_url = "https://github.com/cgohlke/talib-build/releases/download/v0.5.1/TA_Lib-0.5.1-cp311-cp311-win_amd64.whl"
        talib_whl = talib_url.split("/")[-1]  # Nome do arquivo

        # Caminho local para salvar o arquivo .whl
        local_path = os.path.join(os.getcwd(), talib_whl)

        # Fazer o download do arquivo .whl
        if not os.path.exists(local_path):
            print(f"Baixando {talib_whl} de {talib_url}...")
            response = requests.get(talib_url, stream=True)
            if response.status_code == 200:
                with open(local_path, "wb") as f:
                    for chunk in response.iter_content(chunk_size=1024):
                        f.write(chunk)
                print(f"Download concluído: {local_path}")
            else:
                print(f"Erro ao baixar {talib_url}: {response.status_code}")
                return False
        else:
            print(f"O arquivo {talib_whl} já existe. Pulando o download.")

        # Instalar o arquivo .whl com pip
        print(f"Instalando {talib_whl}...")
        try:
            subprocess.check_call(["pip", "install", local_path])
            print("TA-Lib instalado com sucesso!")
        except subprocess.CalledProcessError as e:
            print(f"Erro durante a instalação: {e}")
            return False

        # Remover o arquivo .whl após a instalação (opcional)
        if os.path.exists(local_path):
            os.remove(local_path)
            print(f"Arquivo {local_path} removido após a instalação.")
        return True


# Uso
if __name__ == "__main__":
    installer = TALibInstaller()
    installer.install_ta_lib_windows()


Iniciando instalação do TA-Lib no Windows...
O arquivo TA_Lib-0.5.1-cp311-cp311-win_amd64.whl já existe. Pulando o download.
Instalando TA_Lib-0.5.1-cp311-cp311-win_amd64.whl...
TA-Lib instalado com sucesso!
Arquivo x:\Taurus\market-analyzer\backend\test\TA_Lib-0.5.1-cp311-cp311-win_amd64.whl removido após a instalação.


In [37]:
import yfinance as yf
import talib
import numpy as np
import matplotlib.pyplot as plt

# Baixar dados históricos para um ativo (exemplo: ^GDAXI - DAX)
start, end = '2019-01-01', '2024-07-05'
prices = yf.download('^GDAXI', start=start, end=end)


[*********************100%***********************]  1 of 1 completed


In [40]:
# pattern detction
harami = talib.CDLHARAMI(prices['Open'].values, prices['High'].values, prices['Low'].values, prices['Close'].values)
harami

Exception: input array has wrong dimensions

In [ ]:
# data preparation for plotting
bull_harami_plot = np.where(harami>0, prices.Close,np.nan)
bear_harami_plot = np.where(harami<0, prices.Close,np.nan)

In [ ]:
# PLotting of Data
title = f'DAX between {start} to {end}'

# plotting
fig = go.Figure()
fig.add_trace(go.Candlestick(x=prices.index, open=prices['Open'], high=prices['High'], 
                             low=prices['Low'], close=prices['Close']))
fig.add_trace(go.Scatter(x=prices.index, y=bull_harami_plot, mode='markers', 
                         name='bull_harami', marker_symbol='triangle-up', marker_color='yellow', marker_size=15)) 
fig.add_trace(go.Scatter(x=prices.index, y=bear_harami_plot, mode='markers', 
                         name='bear_harami', marker_symbol='triangle-down', marker_color='lightskyblue', marker_size=15))

fig.update_layout(template='plotly_dark', autosize=False, width=1200, height=600)
fig.update_layout(title=title, xaxis_title='date', yaxis_title='prices')
fig.update_layout(xaxis_rangeslider_visible=False)
fig.show()